In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.utils import shuffle
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

### CV for old features

In [14]:
train_set = pd.read_csv('data/train_nx_ig_v3.csv')
X = train_set.iloc[:,2:-1]
y = train_set.iloc[:,-1]

In [ ]:
X, y = shuffle(X, y, random_state=0)
X.head()

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [19]:
clf_xgb = XGBClassifier(    
                        n_estimators=200,
                        max_depth=3,
                        objective="binary:logistic",
                        learning_rate=.3, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=5,
                        reg_alpha=2,
                        reg_lambda=500,
                     )
np.mean(cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1'))

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.99532123013673

This result is overfitting because of jaccard coefficient

In [20]:
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=5)

[0]	validation_0-logloss:0.471532
Will train until validation_0-logloss hasn't improved in 5 rounds.
[1]	validation_0-logloss:0.326991
[2]	validation_0-logloss:0.237644
[3]	validation_0-logloss:0.187202
[4]	validation_0-logloss:0.152868
[5]	validation_0-logloss:0.128762
[6]	validation_0-logloss:0.101147
[7]	validation_0-logloss:0.082165
[8]	validation_0-logloss:0.067896
[9]	validation_0-logloss:0.058138
[10]	validation_0-logloss:0.05059
[11]	validation_0-logloss:0.045376
[12]	validation_0-logloss:0.041136
[13]	validation_0-logloss:0.039062
[14]	validation_0-logloss:0.03721
[15]	validation_0-logloss:0.03402
[16]	validation_0-logloss:0.031813
[17]	validation_0-logloss:0.031123
[18]	validation_0-logloss:0.029569
[19]	validation_0-logloss:0.029001
[20]	validation_0-logloss:0.027655
[21]	validation_0-logloss:0.026595
[22]	validation_0-logloss:0.025749
[23]	validation_0-logloss:0.024974
[24]	validation_0-logloss:0.024335
[25]	validation_0-logloss:0.02364
[26]	validation_0-logloss:0.023218
[2

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=5, learning_rate=0.3, max_delta_step=0,
       max_depth=3, min_child_weight=6, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=2, reg_lambda=500, scale_pos_weight=1.6, seed=None,
       silent=True, subsample=0.8)

### Train, predict and output

In [21]:
test_set = pd.read_csv('data/test_nx_ig_v3.csv')
X_test = test_set.iloc[:,2:]
y_test = clf_xgb.predict(X_test)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
df = pd.DataFrame(y_test, columns=['category'])
df.index.name = 'id'
df.to_csv('result/xgb3.csv', index=True, header=True)

### with new features of Shiwen

In [2]:
train_set = pd.read_csv('data/train_treated.csv')
train_set.head()

,id1,id2,link,rno1,rno2,sim,year1,year2,year_diff,common_authors,cn,aai,title_overlap
0,9510123,9502114,1,16827,15446,0.064373,1995,1995,0,0,1,0.513898,2
1,9707075,9604178,1,21154,18059,0.021211,1997,1996,1,0,20,4.320366,1
2,9312155,9506142,0,13074,16171,0.017202,1993,1995,-2,0,0,0.000000,0
3,9911255,302165,0,27486,9702,0.012634,1999,2003,-4,0,0,0.000000,0
4,9701033,209076,0,19856,8212,0.059588,1997,2002,-5,0,0,0.000000,0


In [3]:
X = train_set.loc[:,['sim', 'year_diff', 'common_authors', 'cn', 'aai', 'title_overlap']]
y = train_set.loc[:,'link']
X.head()

,sim,year_diff,common_authors,cn,aai,title_overlap
0,0.064373,0,0,1,0.513898,2
1,0.021211,1,0,20,4.320366,1
2,0.017202,-2,0,0,0.000000,0
3,0.012634,-4,0,0,0.000000,0
4,0.059588,-5,0,0,0.000000,0


In [4]:
X, y = shuffle(X, y, random_state=0)
X.head()

,sim,year_diff,common_authors,cn,aai,title_overlap
206307,0.057576,-1,0,0,0.000000,1
545369,0.064932,6,0,15,3.795183,0
304493,0.009646,0,0,0,0.000000,0
42171,0.002130,-2,0,0,0.000000,0
268996,0.095356,0,0,0,0.000000,0


In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [11]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9734158687953569


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9735239789593948


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


This result is not overfitting.

### replace 5 old features with 5 new ones

In [10]:
train1 = pd.read_csv('data/train_treated.csv')
train1.head()

,id1,id2,link,rno1,rno2,sim,year1,year2,year_diff,common_authors,cn,aai,title_overlap
0,9510123,9502114,1,16827,15446,0.064373,1995,1995,0,0,1,0.513898,2
1,9707075,9604178,1,21154,18059,0.021211,1997,1996,1,0,20,4.320366,1
2,9312155,9506142,0,13074,16171,0.017202,1993,1995,-2,0,0,0.000000,0
3,9911255,302165,0,27486,9702,0.012634,1999,2003,-4,0,0,0.000000,0
4,9701033,209076,0,19856,8212,0.059588,1997,2002,-5,0,0,0.000000,0


In [11]:
train2 = pd.read_csv('data/train_nx_ig_v3.csv')
train2.head()

,id1,id2,res_allo_ind,overlap_title,temp_diff,comm_auth,reduced_tfidf_sim,jaccard_coeff,tgt_citation,labels
0,9510123,9502114,0.142857,2,0,0,0.242161,0.176471,8,1
1,9707075,9604178,0.226401,1,1,0,0.116740,0.106796,124,1
2,9312155,9506142,0.000000,0,-2,0,0.067705,0.000000,2,0
3,9911255,302165,0.000000,0,-4,0,0.054771,0.000000,2,0
4,9701033,209076,0.000000,0,-5,0,0.220903,0.000000,2,0


In [12]:
X = pd.concat([train1.loc[:,['title_overlap', 'year_diff','common_authors','sim','cn','aai']], train2.loc[:,['jaccard_coeff','tgt_citation']]], axis=1)
y = train1.loc[:,'link']
X.head()

,title_overlap,year_diff,common_authors,sim,cn,aai,jaccard_coeff,tgt_citation
0,2,0,0,0.064373,1,0.513898,0.176471,8
1,1,1,0,0.021211,20,4.320366,0.106796,124
2,0,-2,0,0.017202,0,0.000000,0.000000,2
3,0,-4,0,0.012634,0,0.000000,0.000000,2
4,0,-5,0,0.059588,0,0.000000,0.000000,2


In [13]:
X, y = shuffle(X, y, random_state=0)
X.head()

,title_overlap,year_diff,common_authors,sim,cn,aai,jaccard_coeff,tgt_citation
206307,1,-1,0,0.057576,0,0.000000,0.000000,8
545369,0,6,0,0.064932,15,3.795183,0.079439,164
304493,0,0,0,0.009646,0,0.000000,0.000000,0
42171,0,-2,0,0.002130,0,0.000000,0.000000,2
268996,0,0,0,0.095356,0,0.000000,0.030303,14


In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [15]:
clf_xgb = XGBClassifier(    
                        n_estimators=200,
                        max_depth=3,
                        objective="binary:logistic",
                        learning_rate=.3, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=5,
                        reg_alpha=2,
                        reg_lambda=5,
                     )
np.mean(cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1'))

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9960641509156408

This result is overfitting because of jaccard coefficient

### Trials of features

#### 6 new features + tgt citation

In [40]:
train1 = pd.read_csv('data/train_treated.csv')
train2 = pd.read_csv('data/train_nx_ig_v2.csv')
X = pd.concat([train1.loc[:,['title_overlap', 'year_diff','common_authors','sim','cn','aai']], train2.loc[:,['tgt_citation']]], axis=1)
y = train1.loc[:,'link']
X, y = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [ ]:
params_range = {'n_estimators':[100,200,300,400,500], 'max_depth':[3,4], 'learning_rate':[0.1,0.2,0.3,0.4], 'subsample':[0.6,0.7,0.8,0.9,1]}
clf_xgb = XGBClassifier(booster='gbtree',objective="binary:logistic",random_state=0)
clf = GridSearchCV(clf_xgb, param_grid=params_range, scoring='f1',cv=5)
clf.fit(X,y)
print (clf.best_score_)
print (clf.best_estimator_)

In [18]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.4, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9750323735926031


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751365966722221


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.3, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9752286153844733


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753423131238748


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9752185936849571


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753518900772937


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9752188923960535


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753581259631107


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


lower learning rate is better

In [22]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751720296182228


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753447668489581


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751846764675779


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.975384521653423


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=.8,
                        colsample_bylevel=.8,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.975158204280947


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753528063249759


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.3, 
                        subsample=.8,
                        colsample_bytree=.8,
                        colsample_bylevel=.8,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751780846533524


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753319678716019


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
clf_xgb = XGBClassifier(    
                        n_estimators=500,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751650045688935


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753250775885473


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [24]:
clf_xgb = XGBClassifier(    
                        n_estimators=300,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=50)
y_pred = clf_xgb.predict(X)
f1_score(y,y_pred)

[0]	validation_0-logloss:0.531966
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.423995
[2]	validation_0-logloss:0.346683
[3]	validation_0-logloss:0.289713
[4]	validation_0-logloss:0.246416
[5]	validation_0-logloss:0.213085
[6]	validation_0-logloss:0.187241
[7]	validation_0-logloss:0.166972
[8]	validation_0-logloss:0.150982
[9]	validation_0-logloss:0.138248
[10]	validation_0-logloss:0.127881
[11]	validation_0-logloss:0.119555
[12]	validation_0-logloss:0.112934
[13]	validation_0-logloss:0.107539
[14]	validation_0-logloss:0.103228
[15]	validation_0-logloss:0.099596
[16]	validation_0-logloss:0.096725
[17]	validation_0-logloss:0.09433
[18]	validation_0-logloss:0.092427
[19]	validation_0-logloss:0.090796
[20]	validation_0-logloss:0.089465
[21]	validation_0-logloss:0.088102
[22]	validation_0-logloss:0.087161
[23]	validation_0-logloss:0.086252
[24]	validation_0-logloss:0.085604
[25]	validation_0-logloss:0.085008
[26]	validation_0-logloss:0.08449

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9757823039220848

In [25]:
test1 = pd.read_csv('data/test_treated.csv')
test2 = pd.read_csv('data/test_nx_ig_v2.csv')
X_test = pd.concat([test1.loc[:,['title_overlap','year_diff','common_authors','sim','cn','aai']], test2.loc[:,['tgt_citation']]], axis=1)

y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('result/xgb_6new+tgt.csv', index=True, header=True)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [41]:
clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=50)
y_pred = clf_xgb.predict(X)
f1_score(y,y_pred)

[0]	validation_0-logloss:0.608258
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.538695
[2]	validation_0-logloss:0.48083
[3]	validation_0-logloss:0.431826
[4]	validation_0-logloss:0.390056
[5]	validation_0-logloss:0.354182
[6]	validation_0-logloss:0.323214
[7]	validation_0-logloss:0.29624
[8]	validation_0-logloss:0.27268
[9]	validation_0-logloss:0.252101
[10]	validation_0-logloss:0.23391
[11]	validation_0-logloss:0.217944
[12]	validation_0-logloss:0.203805
[13]	validation_0-logloss:0.191321
[14]	validation_0-logloss:0.18022
[15]	validation_0-logloss:0.170361
[16]	validation_0-logloss:0.161544
[17]	validation_0-logloss:0.15371
[18]	validation_0-logloss:0.146734
[19]	validation_0-logloss:0.140453
[20]	validation_0-logloss:0.134857
[21]	validation_0-logloss:0.129779
[22]	validation_0-logloss:0.125277
[23]	validation_0-logloss:0.121218
[24]	validation_0-logloss:0.117582
[25]	validation_0-logloss:0.114339
[26]	validation_0-logloss:0.111437
[27

[230]	validation_0-logloss:0.079578
[231]	validation_0-logloss:0.079579
[232]	validation_0-logloss:0.07958
[233]	validation_0-logloss:0.079581
[234]	validation_0-logloss:0.079583
[235]	validation_0-logloss:0.079587
[236]	validation_0-logloss:0.079586
[237]	validation_0-logloss:0.079587
[238]	validation_0-logloss:0.079584
[239]	validation_0-logloss:0.079582
[240]	validation_0-logloss:0.079583
[241]	validation_0-logloss:0.079583
[242]	validation_0-logloss:0.079583
[243]	validation_0-logloss:0.079578
[244]	validation_0-logloss:0.079579
[245]	validation_0-logloss:0.079574
[246]	validation_0-logloss:0.079571
[247]	validation_0-logloss:0.079571
[248]	validation_0-logloss:0.079565
[249]	validation_0-logloss:0.079565
[250]	validation_0-logloss:0.079566
[251]	validation_0-logloss:0.079563
[252]	validation_0-logloss:0.079563
[253]	validation_0-logloss:0.079561
[254]	validation_0-logloss:0.079561
[255]	validation_0-logloss:0.079553
[256]	validation_0-logloss:0.079555
[257]	validation_0-logloss:0.

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9757117353422903

In [42]:
test1 = pd.read_csv('data/test_treated.csv')
test2 = pd.read_csv('data/test_nx_ig_v2.csv')
X_test = pd.concat([test1.loc[:,['title_overlap','year_diff','common_authors','sim','cn','aai']], test2.loc[:,['tgt_citation']]], axis=1)

y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('result/xgb_6new+tgt_2.csv', index=True, header=True)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### 6 new features + tgt citation + rai

In [34]:
train1 = pd.read_csv('data/train_treated.csv')
train2 = pd.read_csv('data/train_nx_ig_v2.csv')
X = pd.concat([train1.loc[:,['title_overlap', 'year_diff','common_authors','sim','cn','aai']], train2.loc[:,['res_allo_ind','tgt_citation']]], axis=1)
y = train1.loc[:,'link']
X, y = shuffle(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

In [29]:
clf_xgb = XGBClassifier(    
                        n_estimators=500,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.2, 
                        subsample=.8,
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        gamma=0,
                        random_state=0
                     )
print (cross_val_score(clf_xgb, X_train, y_train, cv=5, scoring='f1').mean())
print (cross_val_score(clf_xgb, X, y, cv=5, scoring='f1').mean())

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9751800782293341


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.9753527316423991


/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [30]:
params_range = {'n_estimators':[100,200,300,400,500], 'max_depth':[3,4], 'learning_rate':[0.1,0.2,0.3,0.4], 'subsample':[0.6,0.7,0.8,0.9,1]}
clf_xgb = XGBClassifier(booster='gbtree',objective="binary:logistic",random_state=0)
clf = GridSearchCV(clf_xgb, param_grid=params_range, scoring='f1',cv=5)
clf.fit(X,y)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

...

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 4], 'n_estimators': [100, 200, 300, 400, 500], 'learning_rate': [0.1, 0.2, 0.3, 0.4], 'subsample': [0.6, 0.7, 0.8, 0.9, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=0)

In [31]:
print (clf.best_score_)
print (clf.best_estimator_)

0.9754683031584271
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


In [38]:
clf_xgb = XGBClassifier(    
                        n_estimators=400,
                        max_depth=4,
                        booster='gbtree',
                        objective="binary:logistic",
                        learning_rate=.1, 
                        subsample=1,
                        random_state=0
                     )
clf_xgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_valid,y_valid)], early_stopping_rounds=50)
y_pred = clf_xgb.predict(X)
f1_score(y,y_pred)

[0]	validation_0-logloss:0.607958
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.538193
[2]	validation_0-logloss:0.480086
[3]	validation_0-logloss:0.431064
[4]	validation_0-logloss:0.3891
[5]	validation_0-logloss:0.353143
[6]	validation_0-logloss:0.321954
[7]	validation_0-logloss:0.29481
[8]	validation_0-logloss:0.271187
[9]	validation_0-logloss:0.250503
[10]	validation_0-logloss:0.232266
[11]	validation_0-logloss:0.216178
[12]	validation_0-logloss:0.201967
[13]	validation_0-logloss:0.189357
[14]	validation_0-logloss:0.178192
[15]	validation_0-logloss:0.168204
[16]	validation_0-logloss:0.159381
[17]	validation_0-logloss:0.151438
[18]	validation_0-logloss:0.14443
[19]	validation_0-logloss:0.138054
[20]	validation_0-logloss:0.132429
[21]	validation_0-logloss:0.127419
[22]	validation_0-logloss:0.122854
[23]	validation_0-logloss:0.118815
[24]	validation_0-logloss:0.115173
[25]	validation_0-logloss:0.111863
[26]	validation_0-logloss:0.108918
[

[230]	validation_0-logloss:0.079243
[231]	validation_0-logloss:0.079246
[232]	validation_0-logloss:0.079251
[233]	validation_0-logloss:0.07925
[234]	validation_0-logloss:0.079252
[235]	validation_0-logloss:0.079253
[236]	validation_0-logloss:0.079252
[237]	validation_0-logloss:0.079251
[238]	validation_0-logloss:0.079252
[239]	validation_0-logloss:0.079252
[240]	validation_0-logloss:0.079252
[241]	validation_0-logloss:0.079249
[242]	validation_0-logloss:0.079249
[243]	validation_0-logloss:0.079251
[244]	validation_0-logloss:0.079251
[245]	validation_0-logloss:0.079253
[246]	validation_0-logloss:0.079254
[247]	validation_0-logloss:0.079253
[248]	validation_0-logloss:0.079251
[249]	validation_0-logloss:0.079251
[250]	validation_0-logloss:0.079253
[251]	validation_0-logloss:0.079255
[252]	validation_0-logloss:0.079249
[253]	validation_0-logloss:0.079248
[254]	validation_0-logloss:0.07925
[255]	validation_0-logloss:0.079247
[256]	validation_0-logloss:0.079247
[257]	validation_0-logloss:0.0

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9756416755542786

In [39]:
test1 = pd.read_csv('data/test_treated.csv')
test2 = pd.read_csv('data/test_nx_ig_v2.csv')
X_test = pd.concat([test1.loc[:,['title_overlap','year_diff','common_authors','sim','cn','aai']], test2.loc[:,['res_allo_ind','tgt_citation']]], axis=1)

y_test_pred = clf_xgb.predict(X_test)
df = pd.DataFrame(y_test_pred, columns=['category'])
df.index.name = 'id'
df.to_csv('result/xgb_6new+tgt+rai.csv', index=True, header=True)

/home/wangsun/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
